In [1]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import sys
import os
sys.path.append('perteval')
sys.path.append('uq_score')

In [2]:
import json
from utils import get_requests, get_results
from perteval.perteval_end_to_end import get_perteval_results
# from uq_score.uq_end_to_end import run_uq
from llm_uncertainty_bench.uq_end_to_end import run_uq_benchmark
import datetime
import perteval.transition_analysis as tas
import time
import subprocess

In [3]:
# try removing old processes that utilize the gpu
try:
    subprocess.run(["nvidia-smi | grep 'python' | awk '{ print $5 }' | xargs -n1 kill -9"], shell=True)
except:
    pass

kill: not enough arguments


In [4]:
if len(sys.argv) == 1 or sys.argv[1] == '-f':
    # -f is to avoid the case that this runs as ipynb
    all_request_models = get_requests('cc4718/requests')
    all_result_models = get_results('cc4718/results')
    models_todo = set(all_request_models) - set(all_result_models)
else:
    models_todo = [sys.argv[1]]
print(models_todo)

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

/u/modelfactory/.cache/huggingface/hub/datasets--cc4718--requests/snapshots/ed8a000faff905923cadd18a9229d2d1b5036768


Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

/u/modelfactory/.cache/huggingface/hub/datasets--cc4718--results/snapshots/2b7dae5c279fc817670edd2b2eb2774a97b0e415/demo-leaderboard/gpt2-demo
{'ibm-granite/granite-3.1-8b-instruct'}


In [5]:
for model_name in models_todo:
    # perteval
    original_log_path = get_perteval_results(model_name, 'original')
    print('Finished perteval on original data')
    # wait a few seconds to kill vllm to spin up a new
    time.sleep(10)
    perturb_log_path = get_perteval_results(model_name, 'perturb')
    print('Finished perteval on perturbed data')
    original, perturb, consist = tas.transition_analysis(original_log_path, perturb_log_path, subjects=["failure_mode_sensor_analysis"])
    asset_scores = tas.get_record_id_for_correct_answer(original_log_path, dimention='asset_name')
    relevancy_scores = tas.get_record_id_for_correct_answer(original_log_path, dimention='relevancy')
    # uq on the original dataset
    # dataset_path = 'data/fmsr'
    # accuracy, nll_loss_avg, ece_score_avg = run_uq(model_name=model_name, dataset_path=dataset_path)
    # uq bench
    print('Running LLM Uncertainty Bench')
    uq_scores = run_uq_benchmark(model_name)
    result_dict = {
        "config": {
            "model_dtype": "torch.bfloat16", 
            "model_name": model_name,
            "model_sha": "main"
        },
        "results": {
            "acc_overall": {
                "acc": original
            },
            "acc_sel": {
                "acc_sel": relevancy_scores['relevant_sensor_for_failure_mode']
            },
            "acc_el": {
                "acc_el": relevancy_scores['irrelevant_sensor_for_failure_mode']
            },
            "acc_perturb": {
                "perturb_score": perturb
            },
            "score_consistency": {
                "consist_score": consist
            }
        },
    }
    for k, v in uq_scores.items():
        result_dict['results'][k] = {k: v}
    for asset in asset_scores:
        if not asset:
            asset_lower = 'other'
        else:
            asset_lower = asset.lower().replace(' ', '_')
        result_dict['results'][f'acc_{asset_lower}'] = {f'acc_{asset_lower}': asset_scores[asset]}
    out_model_name = model_name.replace('/', '--')
    out_fname = f'results/demo-leaderboard/gpt2-demo/results_{out_model_name}.json'
    with open(out_fname, 'w') as f:
        f.write(json.dumps(result_dict))

waiting for vllm to launch. Retrying in 10 seconds
INFO 05-12 21:09:32 __init__.py:207] Automatically detected platform cuda.
INFO 05-12 21:09:32 api_server.py:912] vLLM API server version 0.7.3
INFO 05-12 21:09:32 api_server.py:913] args: Namespace(subparser='serve', model_tag='ibm-granite/granite-3.1-8b-instruct', config='', host=None, port=8003, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, lora_modules=None, prompt_adapters=None, chat_template=None, chat_template_content_format='auto', response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=False, disable_frontend_multiprocessing=False, enable_request_id_headers=False, enable_auto_tool_choice=False, enable_reasoning=False, reasoning_parser=None, tool_call_parser=None, tool_parser_plugin='', model='ibm-granite/granite-3.1-8b-instruct', task='auto

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:02<00:06,  2.14s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:04<00:04,  2.26s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:05<00:01,  1.58s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:07<00:00,  1.80s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:07<00:00,  1.85s/it]



INFO 05-12 21:09:56 model_runner.py:1115] Loading model weights took 15.2512 GB
waiting for vllm to launch. Retrying in 10 seconds
INFO 05-12 21:09:57 worker.py:267] Memory profiling takes 0.81 seconds
INFO 05-12 21:09:57 worker.py:267] the current vLLM instance can use total_gpu_memory (79.15GiB) x gpu_memory_utilization (0.90) = 71.24GiB
INFO 05-12 21:09:57 worker.py:267] model weights take 15.25GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 55.42GiB.
INFO 05-12 21:09:57 executor_base.py:111] # cuda blocks: 22701, # CPU blocks: 1638
INFO 05-12 21:09:57 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 2.77x
INFO 05-12 21:10:00 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during

Capturing CUDA graph shapes:  40%|████      | 14/35 [00:06<00:09,  2.20it/s]

waiting for vllm to launch. Retrying in 10 seconds


...

INFO:     127.0.0.1:35726 - "POST /v1/chat/completions HTTP/1.1" 200 OK


INFO:httpx:HTTP Request: POST http://localhost:8003/v1/chat/completions "HTTP/1.1 200 OK"
ERROR:root:original_llm_answer = {"option_P": "Piezoelectric sensors do not generate electric charges in response to applied mechanical stress. They require an external power source to function.", "option_Q": "Piezoelectric sensors do require an external power source to operate. This statement is true.", "option_R": "Piezoelectric sensors are indeed useful for cavitation detection. They can measure changes in fluid pressure, temperature, and noise due to cavitation.", "option_S": "Piezoelectric sensors are sensitive to dynamic strain or stress but are not exclusively designed to detect high-frequency vibrations. Their primary use is measuring static and quasi-static strain."}

Answer: ["Q"]

Explanation:

Piezoelectric sensors generate an electric charge in response to applied mechanical stress. They do not require an external power source to function, which means the statement "Piezoelectric sens

INFO:     127.0.0.1:35688 - "POST /v1/chat/completions HTTP/1.1" 200 OK
Finished perteval on perturbed data
========failure_mode_sensor_analysis========
MCQ Benchmark Performance:
score_1 = 0.0 (0/2)
score_2 = 0.0 (0/2)
performance_drop_rate = 0.0
recall_c = 0.0
acc@consist = 0.0
========All (micro)========
MCQ Benchmark Performance:
score_1 = 0.0 (0/2)
score_2 = 0.0 (0/2)
performance_drop_rate = 0.0
recall_c = 0.0
acc@consist = 0.0
========All (macro)========
macro_acc (before) = 0.0
macro_acc (after) = 0.0
macro_pdr = 0.0
macro_rop = 0.0
macro_acc@consist = 0.0
======Incorrect Response Analaysis======
centrifugal pump: 2: 0.5
impeller: 1: 0.0
electric motor: 3: 0.3333333333333333
internal combustion engine: 1: 0.0
reciprocating compressor: 1: 1.0
generator: 1: 1.0
industrial pump: 1: 0.0
gearbox: 1: 1.0
hydraulic system: 1: 0.0
pump: 1: 0.0
piston compressor: 1: 1.0
rotor: 1: 0.0
======Incorrect Response Analaysis======
relevant_component_for_asset: 1: 0.0
relevant_effect_for_failure

INFO:     Shutting down
/u/modelfactory/.conda/envs/lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/u/modelfactory/.conda/envs/lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/u/modelfactory/.conda/envs/lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
INFO:     

granite-3.1-8b-instruct
chat
1
chat logits
cuda
['fmsr.json']
There are 50 data in fmsr.json.
fmsr_Acc: 63.33
Average acc: 63.33
fmsr_SS: 3.47
Average SS: 3.47
fmsr_Coverage Rate: 96.67
Average Coverage Rate: 96.67
fmsr_UAcc: 45.46
Average UAcc: 45.46



In [16]:
# !nvidia-smi | grep 'python' | awk '{ print $5 }' | xargs -n1 kill -9

kill: not enough arguments
